## Examples with ```CopasiProcess``` using ``builder.Builder`` and ``biosimulator_processes.data_model``

In [ ]:
from builder import Builder, Process
from process_bigraph import pp, pf 
from biosimulator_processes import CORE
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder

#### Initialize a fresh instance of the `BiosimulatorBuilder`, which is inherently configured by `biosimulator_processes.CORE`.

In [ ]:
b = BiosimulatorBuilder()

In [ ]:
###### list registered processes

b.list_processes()

In [ ]:
from basico import *
from basico import biomodels


model = load_biomodel('BIOMD0000000861')

In [ ]:
get_species(model=model)

#### Example 1: Add a simple `CopasiProcess`-based composite to the bigraph, specifying parameter values through the `data_model`. For now, let's try loading a biomodel and adding a change to initial concentration of a species

In [ ]:
# 1. specify model changes

process_model_changes = ModelChanges(
    species_changes=SpeciesChanges(
        species_name='SHP1',
        initial_concentration=5.00000
    )
)


pp(process_model_changes.model_dump())

In [ ]:
# 2. define the model schema to be used by the composite process (one of the copasiprocess config parameters)

process_model = Model(
    model_id='simple_copasi_process',
    model_source='BIOMD0000000861',
    model_name='Simple Composite Process',
    model_changes=process_model_changes
)


pp(process_model.model_dump())

In [ ]:
# 3. Define config schema to be used as 'config' parameter of Process constructor


process_config = CopasiProcessConfigSchema(
    method='stochastic',
    model=process_model,
    process_name='simple_copasi'
)


pp(process_config.model_dump())

In [ ]:
# 4. Add the process instance by the name of 'simple_copasi' to the builder


b[process_config.process_name].add_process(
    name='CopasiProcess',
    kdeg=1.0,  # kwargs fill parameters in the config
)

In [ ]:
# visualize shows the process with its disconnected ports
b.visualize()

### print ports

In [ ]:
b['event_process'].interface(True)

### connect ports using connect_all
`Builder.connect_all` connects ports to stores of the same name.

In [ ]:
b.connect_all(append_to_store_name='_store')
b

In [ ]:
b.visualize()

### add interval process to the config

In [ ]:
b['interval_process'].add_process(
    name='GillespieInterval',
)

In [ ]:
b.visualize()

### connect port to specific target

In [ ]:
# to connect a port in a more targeted way, use connect and specify the port and its target path
b['interval_process'].connect(port='interval', target=['event_process', 'interval']) 

# the remaining ports can connect_all
b.connect_all() 

In [ ]:
b.visualize()

### check current Builder config

In [ ]:
b

## Update the initial state

In [ ]:
initial_state = {
    'DNA_store': {
        'A gene': 2.0,
        'B gene': 1.0},
}
b.update(initial_state)

## Generate composite from builder config and simulate

In [ ]:
composite = b.generate()
composite.run(10)

In [ ]:
composite

## Retrieve the composite document

In [ ]:
doc = b.document()
doc

### save the document to file

In [ ]:
b.write(filename='toy_bigraph')

### load a composite from document
This document represents the full state of the composite, and so can reproduce the previous composite when loaded into a fresh Builder

In [ ]:
b2 = Builder(core=core, file_path='out/toy_bigraph.json')
b2

In [ ]:
b2.visualize()

### Example 2: Parameter scan composite